# Demo: RLPF Simulation for Ad Text

This notebook demonstrates a small RLPF-style workflow using either real Ad Library data or synthetic fallback.

**Data source link (not included in the repo):**
- Ad Library API docs: https://developers.facebook.com/docs/marketing-api/reference/ads_archive/

If you have real data, save it as `demo/ads_real.csv` and make sure it contains a text field such as `ad_creative_body`.


In [4]:
import os
import sys

ROOT = os.path.abspath('..')
if ROOT not in sys.path:
    sys.path.insert(0, ROOT)


In [ ]:
import pandas as pd

from src.synthetic_data import generate_synthetic_ads
from src.reward_model import train_reward_model
from src.rlpf_sim import simulate_rlpf

DATA_PATH = 'ads_real.csv'
if os.path.exists(DATA_PATH):
    df = pd.read_csv(DATA_PATH)
else:
    df = pd.DataFrame(generate_synthetic_ads(n=300, seed=42))

df.head()

In [ ]:
df['text'] = df.get('ad_creative_body', df.get('text', '')).fillna('')
df = df[df['text'].str.strip().astype(bool)]

df['score'] = df.get('ctr', 0).fillna(0)
reward_model = train_reward_model(df['text'], df['score'])
results = simulate_rlpf(df['text'].tolist()[:50], reward_model, n_variations=5)
results[:3]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

deltas = np.array([r['delta'] for r in results])
plt.hist(deltas, bins=20)
plt.title('Distribution of Reward Improvements')
plt.xlabel('Delta')
plt.ylabel('Count')
plt.show()